<a href="https://colab.research.google.com/github/swordwalid/ML-Projects-/blob/main/hyperparameter_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner -q

     |████████████████████████████████| 135 kB 24.9 MB/s 
     |████████████████████████████████| 1.6 MB 48.5 MB/s 


In [ ]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
import numpy as np

In [ ]:
df=pd.read_csv('Real_Combine.csv')

In [ ]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [ ]:
for i in range(0,df.shape[0]):
  if np.isnan(df.iloc[i]['PM 2.5']):
    print(i)
  # else:
  #   print("no num")
# df.loc[0,'PM 2.5']
# if (df['PM 2.5']==np.NaN):
#   print('Yes')


184


In [ ]:
# df[pd.to_numeric(df['PM 2.5'],errors='coerce').isnull()]
# df.iloc[184]['PM 2.5']
# df[df['PM 2.5'].isna()]
df.drop(labels=184,axis=0,inplace=True)

In [ ]:
missing_vals=['NA','',None,np.NaN]
missing=df.isin(missing_vals)
missing.sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    0
dtype: int64

In [ ]:
df['PM 2.5'].isnull().sum()

0

In [ ]:
X=df.iloc[:,:-1]
Y=df.iloc[:,-1]

In [ ]:
#creating model
df.shape[0]

1092

In [ ]:

def build_hyp_model(hp):
  model=keras.Sequential()
  for i in range(hp.Int('num_layers',2,20)):
    model.add(layers.Dense(units=hp.Int('units_'+str(i),min_value=30,max_value=100,step=15),
                           activation=hp.Choice('activation',['relu'])))
    model.add(layers.Dense(1,activation='linear'
                           ))
    #learning_rate=hp.Float('lr',min_value=1e-4,max_value=1e-2,sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [ ]:
import keras_tuner
build_hyp_model(keras_tuner.HyperParameters())

In [ ]:
tuner=RandomSearch(hypermodel=build_hyp_model,
                   objective='val_mean_absolute_error',
                   max_trials=10,
                   executions_per_trial=3,
                   directory='my_project',
                   project_name='hyperparameter_ANN')

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 30, 'max_value': 100, 'step': 15, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [ ]:
tuner.search(X_train, y_train,
             epochs=20,
             validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 09s]
val_mean_absolute_error: 64.12756220499675

Best val_mean_absolute_error So Far: 45.82062530517578
Total elapsed time: 00h 01m 29s


In [ ]:
tuner.results_summary()

Results summary
Results in my_project/hyperparameter_ANN
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 15
units_0: 45
activation: relu
learning_rate: 0.01
Score: 45.82062530517578
Trial summary
Hyperparameters:
num_layers: 5
units_0: 75
activation: relu
learning_rate: 0.01
Score: 45.851802825927734
Trial summary
Hyperparameters:
num_layers: 14
units_0: 60
activation: relu
learning_rate: 0.001
Score: 54.47786966959635
Trial summary
Hyperparameters:
num_layers: 7
units_0: 45
activation: relu
learning_rate: 0.001
Score: 55.657118479410805
Trial summary
Hyperparameters:
num_layers: 3
units_0: 75
activation: relu
learning_rate: 0.0001
Score: 63.663673400878906
Trial summary
Hyperparameters:
num_layers: 17
units_0: 60
activation: relu
learning_rate: 0.0001
Score: 64.12756220499675
Trial summary
Hyperparameters:
num_layers: 10
units_0: 75
activation: relu
learning_rate: 0.0001
Score: 64.44696299235027
Trial summary
Hyperparameters:
num_layers: 8
units_0: 45
activation: rel

In [ ]:
models=tuner.get_best_models(num_models=2)
best_model=models[0]
#build model
best_model.build(input_shape=(None,X_train.shape[0],X_train.shape[1] ))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 764, 45)           405       
                                                                 
 dense_1 (Dense)             (None, 764, 1)            46        
                                                                 
Total params: 451
Trainable params: 451
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#If you want to train the model with the entire dataset, you may retrieve the best hyperparameters and retrain the model by yourself.

In [ ]:
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_hyp_model(best_hps[0])
# Fit with the entire dataset.
x_all = pd.concat((X_train, X_test))
y_all = pd.concat((y_train, y_test))
model.fit(x=x_all, y=y_all, epochs=100)


Epoch 1/100
35/35 [==============================] - 0s 2ms/step - loss: 66.1629 - mean_absolute_error: 66.1629
Epoch 2/100
35/35 [==============================] - 0s 1ms/step - loss: 58.0320 - mean_absolute_error: 58.0320
Epoch 3/100
35/35 [==============================] - 0s 2ms/step - loss: 53.0037 - mean_absolute_error: 53.0037
Epoch 4/100
35/35 [==============================] - 0s 2ms/step - loss: 50.5044 - mean_absolute_error: 50.5044
Epoch 5/100
35/35 [==============================] - 0s 2ms/step - loss: 49.0778 - mean_absolute_error: 49.0778
Epoch 6/100
35/35 [==============================] - 0s 2ms/step - loss: 47.6213 - mean_absolute_error: 47.6213
Epoch 7/100
35/35 [==============================] - 0s 1ms/step - loss: 46.7056 - mean_absolute_error: 46.7056
Epoch 8/100
35/35 [==============================] - 0s 2ms/step - loss: 45.5199 - mean_absolute_error: 45.5199
Epoch 9/100
35/35 [==============================] - 0s 1ms/step - loss: 45.3784 - mean_absolute_error: 

[40.78048324584961, 40.78048324584961]

In [ ]:
model.evaluate(X_test, y_test, batch_size=128)

3/3 [==============================] - 0s 4ms/step - loss: 40.7805 - mean_absolute_error: 40.7805


[40.78048324584961, 40.78048324584961]